# Applied Data Science - The Battle of Neighborhoods
The idea is to leverage the Foursquare location data to explore and compare neighborhoods.

## Introduction/Business problem
Below is provided a concise discussion of the business/data science problem and background, and who would be interested in this project.

### Definition of the problem
Someone is looking to open a restaurant in a city of choice, where would we recommend that they open it?

### Where Foursquare location data are needed to solve or execute
Foursquare location data provides information such as the categories of the venues in the neighborhoods of the city of choice. Such data are necessary to analyze the neighborhoods to determine, for example, the most common venues and cluster them. By examining the clusters a determination regarding the discriminating venue categories that distinguish each cluster can be made. Moreover, information such as overall rating, number of tips, agree counts, Foursquare users and users' friends, and popular and trending spots around a venue, are available for any further analysis needed by the stakeholders.

### Description of audience and why they would care about the problem
The target audience for this data science problem are the stakeholders interested in opening a restaurant in the city of choice. The target audience cares about this problem because picking the right neighborhood is of the essence for the success of their business venture.

## Data
Below is provided a concise description of the data that will be used to solve the problem and the source of the data.

### Description of data
The following location data are needed to solve the data science problem:
- Postal codes to determine latitude and longitude coordinates of neighborhoods used to retrieve Foursquare location data.
- Unique categories of the venues in the neighborhoods of the city of choice from Foursquare location data for determining the mean of the frequency of occurrence of each category. Learning most common venues and clustering techniques depend on such data.

### Description of the source of the data
Foursquare is a location data provider. By constructing a specific URL a request can be sent to the Foursquare API to: search for a specific type of venue around a given location, learn more about a specific venue, learn more about a specific Foursquare user, explore a given location, explore trending venues around a given location.

## Methodology
_Discussion and description of performed exploratory data analysis and inferential statistical testing, and machine learnings used and why._

**Not due yet.**

## Results
_Discussion of the results._

**Not due yet.**

## Discussion
_Discussion of noted observations and recommendations that can me made based on the results._

**Not due yet.**

## Conclusions
_Conclusions of the report._

**Not due yet.**

## Presentation/blog post
_Choice of a presentation or blogpost._
- Just like a report, we need to either prepare a presentation of the project or write a blogpost.
- For a presentation, picture yourself presenting the work to a manager or a client, and build your presentation with that mindset.
- For a blogpost, write a summary of the work in a very elegant and intriguing way. Blogposts are one of the means to showcase work done to potential employers and other data scientists as well.

**Not due yet.**